# GridSearchCV em rede neural

In [8]:
import pandas as pd
import numpy as np
import math

#Keras from tensorflow
import keras 
# Rede neural
from tensorflow import keras
#Arquitetura da rede neural
from keras.models import Sequential
#Rede neural fully connected
from keras.layers import Dense
#Import dropout layer from Keras
from keras.layers import Dropout


Using TensorFlow backend.


## Carregar os dados a partir do Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
# Após montar o drive é possível copiar o path a partir do menu à esquerda <=========
!ls '/content/drive/My Drive/DS/DS_Share/Udemy-DeepLearning/S4_ClassificacaoBinaria/colab'

5_Hyperparameters_Tuning.ipynb	entradas.csv  saidas.csv


In [14]:
#Carregar os dados
path = '/content/drive/My Drive/DS/DS_Share/Udemy-DeepLearning/S4_ClassificacaoBinaria/colab'
df_entradas = pd.read_csv(path + '/entradas.csv')
df_saidas = pd.read_csv(path + '/saidas.csv')

In [15]:
X = df_entradas.values
y = df_saidas.values


## Validação cruzada

In [16]:
#O wrapper do keras é uma função da rotina keras responsável por chamar uma segunda subrotina, ou seja, o scikit learn
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score


# GridSearchCV para tuning dos parâmetros


In [17]:
# Pesquisa em grade para encontrar os melhores parâmetros
from sklearn.model_selection import GridSearchCV 

# Criação da rede dentro de uma função (a função retornará o classificador)


In [19]:
# Anteriormente ao GridSearchCV, haviámos criado a rede neural de forma estática. 
# Agora vamos passar como hyperparâmetros:
#optimizer, 
def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):

    #Network creation
    classificador = Sequential()
    
    #Adicao da primeira camada oculta do tipo Dense, ou seja, fully connected
    classificador.add(Dense(units = neurons,
                        activation=activation, #Geralmente bom desempenho para Deep Learning 
                        kernel_initializer=kernel_initializer, #Inicilializacao dos pesos
                        input_dim=30,
                        use_bias = True))
    
    #Camada de dropout introduzida com o propósito de reduzir o overfitting
    #Possui portanto efeito de regularização, assim como ocorre com Ridge e Lasso Regression
    #Introdução de um bias para reduzir a variancia 
    classificador.add(Dropout(rate = 0.2)) #20 % dos neurônios irão zerar
    
    #Adição de mais uma camada oculta:
    classificador.add(Dense(units = neurons,
                        activation=activation, #Geralmente bom desempenho para Deep Learning 
                        kernel_initializer=kernel_initializer,
                        use_bias = True)), #Inicilializacao dos pesos
    
    #Adição da camada de dropout para zerar neurônios da camada anteriormente definida? 
    #Geralmente eh importante se preocupar em reduzir o overfitting de camadas maiores, 
    #Como essas duas camadas bem grandes
    classificador.add(Dropout(rate = 0.2))
    
    #Output layer
    # Para classificação binária, utilizar sigmoid é adequado porque retorna uma probabilidade 
    classificador.add(Dense(units=1, 
                  activation='sigmoid',
                  use_bias = True))    
    
    #Compilacao da rede neural:
    #Classificação binária posso utilizar crossnetropy
    #Antes estávamos utilizando otimizador fixo, e agora será otimizador do GridSearchCV
    classificador.compile(optimizer=optimizer,
                      loss = loss,
                      metrics= ['binary_accuracy']) #Sempre no formato de lista ou dicionário
    
    #Visualizar a arquitetura total da rede mais seus hiperparâmetros definidos para o treinamento
    classificador.summary()
    
    #O importante é que esta funcao retorne o classificador
    return classificador


# Criando o classificador a partir do KerasClassifier

In [20]:
#Na utilização de GridSearchCV, ao instanciar o objeto classificador a partir da classe KerasClassifier, só preciso inserir o build_fn para criar a função 
# e não as épocas e batch size
classificador = KerasClassifier(build_fn=criar_rede)

## Hyperparâmetros que serão testados no tuning

In [21]:
#Hyperparâmetros são inseridos no formato de dicionário. 
#Chaves com o nome do hyperparâmetro e valor na forma de lista
parameters = {'batch_size': [16,32], 
              'epochs': [30,50],
              'optimizer': ['SGD','Adam'],
              'loss': ['binary_crossentropy','poisson'],
              'kernel_initializer': ['random_uniform','random_normal'],
              'activation': ['relu','tanh'],
              'neurons': [16,20]} 

In [22]:
parameters

{'activation': ['relu', 'tanh'],
 'batch_size': [16, 32],
 'epochs': [30, 50],
 'kernel_initializer': ['random_uniform', 'random_normal'],
 'loss': ['binary_crossentropy', 'poisson'],
 'neurons': [16, 20],
 'optimizer': ['SGD', 'Adam']}

# Criação do GridSearchCV
- No próprio GridSearchCV está implementado a Validação Cruzada 

In [23]:
gridsearch = GridSearchCV(estimator = classificador, 
                          param_grid = parameters, 
                          scoring = 'accuracy',
                          cv = 5)

## O treinamento do modelo é realizado a partir do objeto gridsearch instanciado a partir da classe GridSearchCV, logo a esta classe possui a ação, o método, a função fit()

In [24]:
gridsearch = gridsearch.fit(X = X, 
                            y = y)

Streaming output truncated to the last 5000 lines.
Epoch 28/30
455/455 [==============================] - 0s 193us/step - loss: 0.2940 - binary_accuracy: 0.8681
Epoch 29/30
455/455 [==============================] - 0s 195us/step - loss: 0.3043 - binary_accuracy: 0.8703
Epoch 30/30
455/455 [==============================] - 0s 200us/step - loss: 0.2770 - binary_accuracy: 0.8901
Model: "sequential_60"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_178 (Dense)            (None, 20)                620       
_________________________________________________________________
dropout_119 (Dropout)        (None, 20)                0         
_________________________________________________________________
dense_179 (Dense)            (None, 20)                420       
_________________________________________________________________
dropout_120 (Dropout)        (None, 20)                0         
__

KeyboardInterrupt: ignored

In [ ]:
best_params = gridsearch.best_params_
best_accuracy = gridsearch.best_score_  